# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 18 2022 10:48AM,244472,16,EMERSA-8418582,"Emersa Waterbox, LLC",Executing
1,Aug 18 2022 10:47AM,244464,15,30010043,"Alliance Pharma, Inc.",Released
2,Aug 18 2022 10:47AM,244464,15,30010044,"Alliance Pharma, Inc.",Released
3,Aug 18 2022 10:47AM,244464,15,30010045,"Alliance Pharma, Inc.",Released
4,Aug 18 2022 10:47AM,244464,15,30010046,"Alliance Pharma, Inc.",Released
5,Aug 18 2022 10:47AM,244464,15,30010047,"Alliance Pharma, Inc.",Released
6,Aug 18 2022 10:47AM,244464,15,30010048,"Alliance Pharma, Inc.",Released
7,Aug 18 2022 10:47AM,244464,15,30010049,"Alliance Pharma, Inc.",Released
8,Aug 18 2022 10:47AM,244464,15,30010050,"Alliance Pharma, Inc.",Released
9,Aug 18 2022 10:47AM,244464,15,30010052,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
51,244468,Released,3
52,244469,Released,1
53,244470,Released,1
54,244471,Released,1
55,244472,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244468,NaN,NaN,3.0
244469,NaN,NaN,1.0
244470,NaN,NaN,1.0
244471,NaN,NaN,1.0
244472,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244330,18.0,2.0,0.0
244357,9.0,2.0,2.0
244370,0.0,0.0,1.0
244385,11.0,4.0,0.0
244396,0.0,0.0,73.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244330,18,2,0
244357,9,2,2
244370,0,0,1
244385,11,4,0
244396,0,0,73


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244330,18,2,0
1,244357,9,2,2
2,244370,0,0,1
3,244385,11,4,0
4,244396,0,0,73


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244330,18,2,
1,244357,9,2,2
2,244370,,,1
3,244385,11,4,
4,244396,,,73


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 18 2022 10:48AM,244472,16,"Emersa Waterbox, LLC"
1,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc."
37,Aug 18 2022 10:44AM,244471,19,"Graystone, LLC"
38,Aug 18 2022 10:36AM,244470,10,ISDIN Corporation
39,Aug 18 2022 10:35AM,244468,10,"Methapharm, Inc."
42,Aug 18 2022 10:35AM,244467,10,"Methapharm, Inc."
56,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc."
57,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc."
58,Aug 18 2022 10:29AM,244465,21,"NBTY Global, Inc."
59,Aug 18 2022 10:26AM,244463,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 18 2022 10:48AM,244472,16,"Emersa Waterbox, LLC",,1,
1,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",,,36
2,Aug 18 2022 10:44AM,244471,19,"Graystone, LLC",,,1
3,Aug 18 2022 10:36AM,244470,10,ISDIN Corporation,,,1
4,Aug 18 2022 10:35AM,244468,10,"Methapharm, Inc.",,,3
5,Aug 18 2022 10:35AM,244467,10,"Methapharm, Inc.",,,14
6,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc.",,,1
7,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc.",,,1
8,Aug 18 2022 10:29AM,244465,21,"NBTY Global, Inc.",,,1
9,Aug 18 2022 10:26AM,244463,10,Eywa Pharma Inc.,,,29


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 10:48AM,244472,16,"Emersa Waterbox, LLC",,1,
1,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",36,,
2,Aug 18 2022 10:44AM,244471,19,"Graystone, LLC",1,,
3,Aug 18 2022 10:36AM,244470,10,ISDIN Corporation,1,,
4,Aug 18 2022 10:35AM,244468,10,"Methapharm, Inc.",3,,
5,Aug 18 2022 10:35AM,244467,10,"Methapharm, Inc.",14,,
6,Aug 18 2022 10:32AM,244469,22,"NBTY Global, Inc.",1,,
7,Aug 18 2022 10:29AM,244466,10,"Methapharm, Inc.",1,,
8,Aug 18 2022 10:29AM,244465,21,"NBTY Global, Inc.",1,,
9,Aug 18 2022 10:26AM,244463,10,Eywa Pharma Inc.,29,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 10:48AM,244472,16,"Emersa Waterbox, LLC",,1,
1,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",36,,
2,Aug 18 2022 10:44AM,244471,19,"Graystone, LLC",1,,
3,Aug 18 2022 10:36AM,244470,10,ISDIN Corporation,1,,
4,Aug 18 2022 10:35AM,244468,10,"Methapharm, Inc.",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 10:48AM,244472,16,"Emersa Waterbox, LLC",NaN,1.0,NaN
1,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",36.0,NaN,NaN
2,Aug 18 2022 10:44AM,244471,19,"Graystone, LLC",1.0,NaN,NaN
3,Aug 18 2022 10:36AM,244470,10,ISDIN Corporation,1.0,NaN,NaN
4,Aug 18 2022 10:35AM,244468,10,"Methapharm, Inc.",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 18 2022 10:48AM,244472,16,"Emersa Waterbox, LLC",0.0,1.0,0.0
1,Aug 18 2022 10:47AM,244464,15,"Alliance Pharma, Inc.",36.0,0.0,0.0
2,Aug 18 2022 10:44AM,244471,19,"Graystone, LLC",1.0,0.0,0.0
3,Aug 18 2022 10:36AM,244470,10,ISDIN Corporation,1.0,0.0,0.0
4,Aug 18 2022 10:35AM,244468,10,"Methapharm, Inc.",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4644371,145.0,10.0,9.0
102,244450,1.0,0.0,0.0
12,488870,1.0,14.0,0.0
15,244464,36.0,0.0,0.0
16,977759,21.0,1.0,0.0
17,488861,2.0,0.0,0.0
19,977705,21.0,11.0,18.0
20,1466526,102.0,7.0,18.0
21,1466560,6.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4644371,145.0,10.0,9.0
1,102,244450,1.0,0.0,0.0
2,12,488870,1.0,14.0,0.0
3,15,244464,36.0,0.0,0.0
4,16,977759,21.0,1.0,0.0
5,17,488861,2.0,0.0,0.0
6,19,977705,21.0,11.0,18.0
7,20,1466526,102.0,7.0,18.0
8,21,1466560,6.0,0.0,0.0
9,22,244469,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,145.0,10.0,9.0
1,102,1.0,0.0,0.0
2,12,1.0,14.0,0.0
3,15,36.0,0.0,0.0
4,16,21.0,1.0,0.0
5,17,2.0,0.0,0.0
6,19,21.0,11.0,18.0
7,20,102.0,7.0,18.0
8,21,6.0,0.0,0.0
9,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,145.0
1,102,Released,1.0
2,12,Released,1.0
3,15,Released,36.0
4,16,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,17,19,20,21,22
Status,,,,,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0
Executing,10.0,0.0,14.0,0.0,1.0,0.0,11.0,7.0,0.0,0.0
Released,145.0,1.0,1.0,36.0,21.0,2.0,21.0,102.0,6.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0
1,Executing,10.0,0.0,14.0,0.0,1.0,0.0,11.0,7.0,0.0,0.0
2,Released,145.0,1.0,1.0,36.0,21.0,2.0,21.0,102.0,6.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,17,19,20,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0,18.0,18.0,0.0,0.0
1,Executing,10.0,0.0,14.0,0.0,1.0,0.0,11.0,7.0,0.0,0.0
2,Released,145.0,1.0,1.0,36.0,21.0,2.0,21.0,102.0,6.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()